In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
pd.options.display.max_rows = 250

#import plotly.express as px
#import seaborn as sns

**Trying a basic model with Random Forest ## Linear Regression**

We are going to train/ test and split diffferent year ranges

**Massey Rankings go back to 2003 so we have that above to match our features w aggregated season data**

We are going to feauture Aggregated Season Data into WTeamID Stats and LTeamID Stats
and
We are going to use Ordinal Massey Ranking - 
It is one of the best basketball team ranking methods readily available 
(https://masseyratings.com/cb/compare.htm) 
We have an updated 3-12-2022 NMasseyOrdinals up to day 128[3] of the tournament and the NMasseyOrdinals[14] up to tourney date**


**We are going to train our data using these two features for now and hopefully we can add Strength of Schedule or Conference metric**

In [10]:
files = glob.glob('../mens-march-mania-2022/MDataFiles_Stage2/*')
[print(i,'\m/',f) for i,f in enumerate(files)];

0 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneyDetailedResults.csv
1 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneyCompactResults.csv
2 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MSeasons.csv
3 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MMasseyOrdinals_thruDay128.csv
4 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonDetailedResults.csv
5 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneySlots.csv
6 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MGameCities.csv
7 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MConferenceTourneyGames.csv
8 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/Cities.csv
9 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonCompactResults.csv
10 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneySeedRoundSlots.csv
11 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MTeamConferences.csv
12 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MSampleSubmissionStage2.csv
13 \m/ ../mens-march-mania-20

In [11]:
# 6) Use most updated ../mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonCompactResults file to retrieve season results
## 03/12/2022 - Aggregate Stats into WTeamID Stats/ LTeamIDStats

SeasonResults = pd.read_csv(files[4])
SeasonResults = SeasonResults.copy()
SeasonResults = SeasonResults[SeasonResults['Season'] >= 2010].reset_index(drop=True)  
#SeasonResults

winning_cols = ['Season', 'WTeamID', 'WLoc', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']
losing_cols = ['Season', 'LTeamID', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']
columns = ['Season', 'TeamID', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']

winning_stats = SeasonResults[winning_cols].groupby(['Season', 'WTeamID']).mean().reset_index()
winning_stats.columns = columns

losing_stats = SeasonResults[losing_cols].groupby(['Season', 'LTeamID']).mean().reset_index()
losing_stats.columns = columns

reg_season_stats = pd.concat([winning_stats, losing_stats]).groupby(['Season', 'TeamID']).mean().reset_index()

reg_season_stats['id'] = reg_season_stats.Season.astype(str) + reg_season_stats.TeamID.astype(str)


reg_season_stats.head()
#reg_season_stats.to_csv('GroupProj_RegSeas_DF_N.csv', encoding='utf-8', index=False)

# 2003 - 2022

,Season,TeamID,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,id
0,2010,1102,21.238095,45.985119,6.086310,17.877976,11.160714,17.178571,7.220238,21.065476,13.776786,12.470238,4.803571,1.627976,17.148810,20101102
1,2010,1103,24.386957,58.473913,6.180435,19.652174,13.806522,21.028261,14.173913,21.934783,12.550000,13.563043,6.202174,3.102174,20.393478,20101103
2,2010,1104,24.633333,56.062745,5.464706,15.647059,13.488235,19.080392,12.019608,22.992157,13.276471,12.807843,7.754902,3.907843,18.323529,20101104
3,2010,1105,22.066667,57.600000,3.954167,13.258333,17.762500,27.691667,14.462500,22.662500,10.700000,15.962500,9.416667,5.575000,20.791667,20101105
4,2010,1106,21.405128,53.543590,5.953846,19.005128,15.812821,24.433333,13.420513,22.535897,12.805128,16.220513,7.515385,4.207692,20.633333,20101106


In [12]:
# # # 2012 - 2022
# massey_ratings = pd.read_csv(files[3])
# massey_ratings.drop('SystemName', axis=1, inplace=True)
# #massey_ratings = massey_ratings.groupby(['Season', 'TeamID'])

# #massey_ratings = massey_ratings.reset_index()
# massey_ratings['id'] = massey_ratings.Season.astype(str) + massey_ratings.TeamID.astype(str)

# massey_ratings = massey_ratings[massey_ratings['Season'] >= 2012].reset_index(drop=True) 
# massey_ratings.head()

In [16]:
ranks = pd.read_csv(files[14])
ranks = ranks[ranks['Season'] >= 2010].reset_index(drop=True)  


aggregate_ranks = ranks.groupby(['Season', 'TeamID']).agg({'OrdinalRank': ['mean', 'min', 'max']})
aggregate_ranks.columns = ['_'.join(col) for col in aggregate_ranks.columns]

massey_Ranks = aggregate_ranks.reset_index()
massey_Ranks['id'] = massey_Ranks.Season.astype(str) + massey_Ranks.TeamID.astype(str)
massey_Ranks.head()

,Season,TeamID,OrdinalRank_mean,OrdinalRank_min,OrdinalRank_max,id
0,2010,1102,224.586854,59,323,20101102
1,2010,1103,110.380282,70,223,20101103
2,2010,1104,78.136150,44,147,20101104
3,2010,1105,327.139280,203,347,20101105
4,2010,1106,302.364632,220,346,20101106


In [17]:
# 9) Extract tourney data

TourneyCompactResults = pd.read_csv(files[1])
TourneyCompactResults = TourneyCompactResults[TourneyCompactResults['Season'] >= 2010].reset_index(drop=True) 
TourneyCompactResults

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,2010,134,1115,61,1457,44,N,0
1,2010,136,1124,68,1358,59,N,0
2,2010,136,1139,77,1431,59,N,0
3,2010,136,1140,99,1196,92,N,2
4,2010,136,1242,90,1250,74,N,0
...,...,...,...,...,...,...,...,...
728,2021,148,1211,85,1425,66,N,0
729,2021,148,1417,51,1276,49,N,0
730,2021,152,1124,78,1222,59,N,0
731,2021,152,1211,93,1417,90,N,1


# Combine Features

In [19]:
def is_winning(wteam, lteam):
    if wteam < lteam:
        return 1
    else:
        return 0
    
    
march_madness = pd.read_csv(files[1])
march_madness = march_madness[march_madness['Season'] >= 2005].reset_index(drop=True) 
train = march_madness

train['is_win'] = train.apply(lambda x: is_winning(x['WTeamID'], x['LTeamID']), axis=1)
train['team_a'] = train.Season.astype(str) + train.WTeamID.astype(str)
train['team_b'] = train.Season.astype(str) + train.LTeamID.astype(str)

train = train.drop(['WScore', 'LScore'], axis=1)
train = pd.merge(train, massey_Ranks, left_on='team_a', right_on='id').merge(massey_Ranks, left_on='team_b', right_on='id', suffixes=('_teama', '_teamb'))

train = train.drop(['Season_x', 'Season_y', 'id_teama', 'id_teamb', 'TeamID_teama', 'TeamID_teamb'], axis=1)
train = pd.merge(train, reg_season_stats, left_on='team_a', right_on='id').merge(reg_season_stats, left_on='team_b', right_on='id', suffixes=('_teama', '_teamb'))

train = train.drop(['Season_x', 'Season_y', 'team_a', 'team_b', 'TeamID_teama', 'TeamID_teamb', 'id_teama', 'id_teamb'], axis=1)
train = train.drop(['DayNum', 'WTeamID', 'LTeamID', 'NumOT', 'Season', 'WLoc'], axis=1)

print(train.shape)
train.count()

(667, 33)


is_win                    667
OrdinalRank_mean_teama    667
OrdinalRank_min_teama     667
OrdinalRank_max_teama     667
OrdinalRank_mean_teamb    667
OrdinalRank_min_teamb     667
OrdinalRank_max_teamb     667
FGM_teama                 667
FGA_teama                 667
FGM3_teama                667
FGA3_teama                667
FTM_teama                 667
FTA_teama                 667
OR_teama                  667
DR_teama                  667
Ast_teama                 667
TO_teama                  667
Stl_teama                 667
Blk_teama                 667
PF_teama                  667
FGM_teamb                 667
FGA_teamb                 667
FGM3_teamb                667
FGA3_teamb                667
FTM_teamb                 667
FTA_teamb                 667
OR_teamb                  667
DR_teamb                  667
Ast_teamb                 667
TO_teamb                  667
Stl_teamb                 667
Blk_teamb                 667
PF_teamb                  667
dtype: int